In [1]:
!module load devel/cuda/12.4
!export CUDA_LAUNCH_BLOCKING=1


The following have been reloaded with a version change:
  1) devel/cuda/11.8 => devel/cuda/12.4



In [3]:
from utils import load_config
import os
# print(os.getcwd())
os.chdir('/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training')
# print(os.getcwd())

In [4]:
import yaml
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.set_device(0)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import sys
# os.environ["CUDA_VISIBLE_DEVICES"] = "" 
sys.argv = ["train.py"]
from train import init_seed, prepare_training_data, prepare_testing_data, choose_optimizer, choose_scheduler, choose_metric
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from datetime import timedelta
from detectors import DETECTOR
# from trainer.trainer import Trainer

base_path = '/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/logs/training/'
pretrained_paths = ['resnet34_bcos_v2_minimal_2025-03-03-20-04-42/test/avg/ckpt_best.pth',
                    'resnet34_bcos_v2_minimal_2025-03-03-16-08-52/test/avg/ckpt_best.pth',
                    'resnet34_bcos_v2_minimal_2025-03-03-15-05-37/test/avg/ckpt_best.pth',
                    'resnet34_bcos_v2_minimal_2025-03-03-14-18-59/test/avg/ckpt_best.pth',
                    'resnet34_bcos_v2_minimal_2025-03-03-14-00-05/test/avg/ckpt_best.pth'
                   ]
pretrained_paths = [f'{base_path}{pretrained_path}' for pretrained_path in pretrained_paths]
path = "./config/detector/resnet34_bcos_v2_minimal.yaml"
additional_args = {'test_batchSize': 4,
                  'pretrained': pretrained_paths[0]}
config = load_config(path, additional_args=additional_args)

2025-03-04 18:37:26.364226: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 18:37:26.414027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-04 18:37:26.414070: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-04 18:37:26.414093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 18:37:26.422418: I tensorflow/core/platform/cpu_feature_g

In [5]:
from test import test_epoch, test_one_dataset, test_epoch, inference
from tqdm import tqdm
import random
# init seed
# init_seed(config)

torch.manual_seed(34)
random.seed(34)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for strict reproducibility

# prepare the testing data loader
test_data_loaders = prepare_testing_data(config)

# prepare the model (detector)
model_class = DETECTOR[config['model_name']]
model = model_class(config)
for path in pretrained_paths:
    state_dict = torch.load(path)
    # Remove "module." prefix
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_key = k.replace("module.", "")  # Remove "module." prefix
        new_state_dict[new_key] = v
    
    model.load_state_dict(new_state_dict)
    model.cuda()#.to("cuda:0")
    print(next(model.parameters()).device)
    
    model.eval()
    # testing for all test data
    keys = test_data_loaders.keys()
    for key in keys:
        print(key)
        data_dict = test_data_loaders[key].dataset.data_dict
        for i, data_dict in tqdm(enumerate(test_data_loaders[key]),total=len(test_data_loaders[key])):
            if i < 3:
                if 'label_spe' in data_dict:
                    data_dict.pop('label_spe')  # remove the specific label
                data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
                # move data to GPU elegantly
                for key in data_dict.keys():
                    if data_dict[key]!=None:
                        data_dict[key]=data_dict[key].cuda()
                # model forward without considering gradient computation
                # print(data_dict)
                with torch.no_grad():
                    predictions = inference(model, data_dict)
                print(data_dict['label'])
                print(predictions['prob'])
            else:
                break

cuda:0
FaceForensics++


  0%|          | 0/5586 [00:00<?, ?it/s]

tensor([1, 1, 1, 0], device='cuda:0')


  0%|          | 2/5586 [00:07<4:50:52,  3.13s/it] 

tensor([0.7749, 0.5005, 0.6887, 0.8450], device='cuda:0')
tensor([0, 1, 1, 1], device='cuda:0')
tensor([0.7421, 0.6160, 0.9278, 0.9470], device='cuda:0')


...

  0%|          | 3/5586 [00:00<18:25,  5.05it/s]

tensor([1, 1, 1, 0], device='cuda:0')
tensor([0.8400, 0.7242, 0.8006, 0.8611], device='cuda:0')
tensor([0, 1, 1, 1], device='cuda:0')
tensor([0.8533, 0.7098, 0.8778, 0.8425], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([0.8548, 0.9085, 0.6962, 0.8236], device='cuda:0')


cuda:0
FaceForensics++


  0%|          | 3/5586 [00:00<09:54,  9.40it/s]

tensor([1, 1, 1, 0], device='cuda:0')
tensor([0.7881, 0.4953, 0.6724, 0.8533], device='cuda:0')
tensor([0, 1, 1, 1], device='cuda:0')
tensor([0.7623, 0.5628, 0.9402, 0.9453], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([0.8210, 0.9239, 0.6069, 0.7048], device='cuda:0')


## Heatmaps

In [8]:
from bcos.interpretability import grad_to_img, to_numpy
import numpy as np
import matplotlib.pyplot as plt
from test import test_epoch, test_one_dataset, test_epoch, inference
from tqdm import tqdm
import random
# init seed
# init_seed(config)

torch.manual_seed(2)
random.seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for strict reproducibility

device = torch.device("cpu")

# prepare the training data loader
# train_data_loader = prepare_training_data(config)
config['test_batchSize'] = 2
# prepare the testing data loader
test_data_loaders = prepare_testing_data(config)

# prepare the model (detector)
model_class = DETECTOR[config['model_name']]
model = model_class(config)
for path in pretrained_paths:
    state_dict = torch.load(path)
    # Remove "module." prefix
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_key = k.replace("module.", "")  # Remove "module." prefix
        new_state_dict[new_key] = v
    
    model.load_state_dict(new_state_dict)
    model.to(device)#.cuda()
    print(next(model.parameters()).device)
    # testing for all test data
    key = list(test_data_loaders.keys())[0]
    print(key)
    for i, data_dict in enumerate(test_data_loaders[key]):#img_batch, label_batch in dataloader:
        if i<3:
            print(data_dict.keys())
            img_batch, label_batch, landmark, mask = (data_dict[k] for k in ['image', 'label', 'landmark', 'mask'])
    
            print(f"Batch of images shape: {img_batch.shape}")
            # device = "cpu"
            if 'label_spe' in data_dict:
                data_dict.pop('label_spe')  # remove the specific label
            data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
            # move data to GPU elegantly
            for key in data_dict.keys():
                if data_dict[key]!=None:
                    data_dict[key]=data_dict[key].to(device)#cuda()
    
            num_batches = img_batch.shape[0]
            # Iterate through each image in the batch
            for i in range(num_batches):
                img = img_batch[i].unsqueeze(0).to(device)#).cuda()  # Process a single image
                label = label_batch[i]
        
                img = img.requires_grad_(True)
        
                # Zero the gradients
                model.zero_grad()
        
                single_data_dict = {'image':img, 'label':label}
                # Forward pass
                features = model.features(single_data_dict)
                # get the prediction by classifier
                out = model.classifier(features)
    
                # we have to select a target class for which we compute the gradients 
                # -> simply choose the first one, as it represents the probability that a image is a deepfake
                print(out)
                print(torch.softmax(out, dim=0))
                prob = torch.softmax(out, dim=0)[1]
                out = out[1]
                # Backward pass
                out.backward()
        
                # Generate attention visualization
                att = grad_to_img(img[0], img.grad[0], alpha_percentile=100, smooth=5)
                att[..., -1] *= to_numpy(out.sigmoid())
        
                # Prepare the image and attention map for visualization
                att = to_numpy(att)
                print(img.shape)
                print("R", img[0, 1, :, :])
                print(img[0].shape)
                # print(img[1].shape)
                # print(img[2].shape)
                img_np = np.array(to_numpy(img[0, [0, 1, 2]].permute(1, 2, 0)) * 255, dtype=np.uint8)
        
                # Plot the results
                fig, ax = plt.subplots(1, figsize=(8, 4))
                plt.imshow(img_np, extent=(0, config['resolution'], 0, config['resolution']))
                plt.imshow(att, extent=(config['resolution'], 2 * config['resolution'], 0, config['resolution']))
                plt.xlim(0, 2 * config['resolution'])
                plt.xticks([])
                plt.yticks([])
                plt.title(f"True Label: {label}, Predictions: {out.sigmoid().item():.2f}")
                for spine in ax.spines.values():
                    spine.set_visible(False)
        
                plt.show()
        else:
            break  # Exit after processing the first batches


cpu
FaceForensics++
dict_keys(['image', 'label', 'landmark', 'mask'])
Batch of images shape: torch.Size([2, 6, 224, 224])
tensor([[ 0.2857, -0.3205]], grad_fn=<AddBackward0>)
tensor([[1., 1.]], grad_fn=<SoftmaxBackward0>)


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [21]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set model to evaluation mode
model.eval()

# Lists to store all predictions and true labels
all_preds = []
all_labels = []

# Disable gradient calculation
with torch.no_grad():
    key = list(test_data_loaders.keys())[0]
    # print(key)
    prediction_lists = []
    feature_lists = []
    label_lists = []
    for i, data_dict in tqdm(enumerate(test_data_loaders[key]), total=len(test_data_loaders[key])):#img_batch, label_batch in dataloader:
        img_batch, label_batch, landmark, mask = (data_dict[k] for k in ['image', 'label', 'landmark', 'mask'])

        # print(f"Batch of images shape: {img_batch.shape}")
        # device = "cpu"
        if 'label_spe' in data_dict:
            data_dict.pop('label_spe')  # remove the specific label
        data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
        # move data to GPU elegantly
        for key in data_dict.keys():
            if data_dict[key]!=None:
                data_dict[key]=data_dict[key].to(device)#cuda()


        predictions = inference(model, data_dict)
        cls, prob, feat = (predictions[k] for k in ['cls', 'prob', 'feat'])
        
        label_lists += list(data_dict['label'].cpu().detach().numpy())
        prediction_lists += list(predictions['prob'].cpu().detach().numpy())
        feature_lists += list(predictions['feat'].cpu().detach().numpy())

    # print(prediction_lists)
    y_pred = np.array(prediction_lists)
    threshold = 0.5
    y_pred = (y_pred >= threshold).astype(int)
    y_true = np.array(label_lists)
    # print(y_true)

100%|██████████| 11171/11171 [04:28<00:00, 41.67it/s]


In [51]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculate and print the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Test Accuracy: {accuracy:.2f}')

# Detailed classification report
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
print(confusion_matrix(y_true, y_pred))

Test Accuracy: 0.80
              precision    recall  f1-score   support

        Real       0.18      0.01      0.01      4478
        Fake       0.80      0.99      0.89     17864

    accuracy                           0.80     22342
   macro avg       0.49      0.50      0.45     22342
weighted avg       0.68      0.80      0.71     22342

[[   29  4449]
 [  128 17736]]


# XCeption net

In [29]:
path = "./config/detector/xception.yaml"

weights_base_path = '/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/'
additional_args = {'test_batchSize': 12, 
                   'pretrained':
                    f'{weights_base_path}xception_2025-02-06-20-05-09/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-10-17-17/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-12-19-31/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-17-23-41/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-26-03/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-43-33/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-08-09-13-00/test/avg/ckpt_best.pth'
                  }
config = load_config(path, additional_args=additional_args)

In [30]:
from test import test_epoch, test_one_dataset, test_epoch, inference
from tqdm import tqdm
import random
# init seed
# init_seed(config)

torch.manual_seed(34)
random.seed(34)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for strict reproducibility

# prepare the testing data loader
test_data_loaders = prepare_testing_data(config)

# prepare the model (detector)
model_class = DETECTOR[config['model_name']]
model = model_class(config)

model.cuda()#.to("cuda:0")
print(next(model.parameters()).device)

model.eval()
# testing for all test data
keys = test_data_loaders.keys()
for key in keys:
    print(key)
    data_dict = test_data_loaders[key].dataset.data_dict
    for i, data_dict in tqdm(enumerate(test_data_loaders[key]),total=len(test_data_loaders[key])):
        if i < 3:
            if 'label_spe' in data_dict:
                data_dict.pop('label_spe')  # remove the specific label
            data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
            # move data to GPU elegantly
            for key in data_dict.keys():
                if data_dict[key]!=None:
                    data_dict[key]=data_dict[key].cuda()
            # model forward without considering gradient computation
            # print(data_dict)
            with torch.no_grad():
                predictions = inference(model, data_dict)
            print(data_dict['label'])
            print(predictions['prob'])
        else:
            break

INFO:detectors.xception_detector:Load pretrained model successfully!


cuda:0
FaceForensics++


  0%|          | 3/1862 [00:00<02:39, 11.68it/s]

tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0.5063, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062,
        0.5062, 0.5062, 0.5062], device='cuda:0')
tensor([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1], device='cuda:0')
tensor([0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5063, 0.5062,
        0.5063, 0.5062, 0.5062], device='cuda:0')
tensor([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1], device='cuda:0')
tensor([0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062, 0.5062,
        0.5062, 0.5062, 0.5062], device='cuda:0')


In [31]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set model to evaluation mode
model.eval()

# Lists to store all predictions and true labels
all_preds = []
all_labels = []

# Disable gradient calculation
with torch.no_grad():
    key = list(test_data_loaders.keys())[0]
    # print(key)
    prediction_lists = []
    feature_lists = []
    label_lists = []
    for i, data_dict in tqdm(enumerate(test_data_loaders[key]), total=len(test_data_loaders[key])):#img_batch, label_batch in dataloader:
        img_batch, label_batch, landmark, mask = (data_dict[k] for k in ['image', 'label', 'landmark', 'mask'])

        # print(f"Batch of images shape: {img_batch.shape}")
        # device = "cpu"
        if 'label_spe' in data_dict:
            data_dict.pop('label_spe')  # remove the specific label
        data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
        # move data to GPU elegantly
        for key in data_dict.keys():
            if data_dict[key]!=None:
                data_dict[key]=data_dict[key].to(device)#cuda()


        predictions = inference(model, data_dict)
        cls, prob, feat = (predictions[k] for k in ['cls', 'prob', 'feat'])
        
        label_lists += list(data_dict['label'].cpu().detach().numpy())
        prediction_lists += list(predictions['prob'].cpu().detach().numpy())
        feature_lists += list(predictions['feat'].cpu().detach().numpy())

    # print(prediction_lists)
    y_pred = np.array(prediction_lists)
    threshold = 0.5
    y_pred = (y_pred >= threshold).astype(int)
    y_true = np.array(label_lists)
    # print(y_true)

100%|██████████| 1862/1862 [01:14<00:00, 24.97it/s]


In [28]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculate and print the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Test Accuracy: {accuracy:.2f}')

# Detailed classification report
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
print(confusion_matrix(y_true, y_pred))

Test Accuracy: 0.80
              precision    recall  f1-score   support

        Real       0.00      0.00      0.00      4478
        Fake       0.80      1.00      0.89     17864

    accuracy                           0.80     22342
   macro avg       0.40      0.50      0.44     22342
weighted avg       0.64      0.80      0.71     22342

[[    0  4478]
 [    0 17864]]


# Evaluating multiple model runs

## BCOS RESNET

In [ ]:
# check performance of all model runs
weights_base_path = '/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/resnet_from_scratch_runs/logs/'
runs = [f'{weights_base_path}resnet34_bcos_2025-02-05-14-05-08/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-05-16-31-48/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-05-18-32-54/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-06-19-46-26/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-07-09-30-34/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-07-12-02-12/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-07-17-02-26/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-07-21-26-03/test/avg/ckpt_best.pth',
        f'{weights_base_path}resnet34_bcos_2025-02-08-09-15-43/test/avg/ckpt_best.pth']              

for model_run in runs:
    path = "./config/detector/resnet34_bcos.yaml"
    
    additional_args = {'test_batchSize': 12, 
                       'pretrained':model_run}
    config = load_config(path, additional_args=additional_args)
    test_data_loaders = prepare_testing_data(config)

    # prepare the model (detector)
    model_class = DETECTOR[config['model_name']]
    model = model_class(config)
    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Lists to store all predictions and true labels
    all_preds = []
    all_labels = []
    
    # Disable gradient calculation
    with torch.no_grad():
        key = list(test_data_loaders.keys())[0]
        # print(key)
        prediction_lists = []
        feature_lists = []
        label_lists = []
        for i, data_dict in tqdm(enumerate(test_data_loaders[key]), total=len(test_data_loaders[key])):#img_batch, label_batch in dataloader:
            img_batch, label_batch, landmark, mask = (data_dict[k] for k in ['image', 'label', 'landmark', 'mask'])
    
            # print(f"Batch of images shape: {img_batch.shape}")
            # device = "cpu"
            if 'label_spe' in data_dict:
                data_dict.pop('label_spe')  # remove the specific label
            data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
            # move data to GPU elegantly
            for key in data_dict.keys():
                if data_dict[key]!=None:
                    data_dict[key]=data_dict[key].to(device)#cuda()
    
    
            predictions = inference(model, data_dict)
            cls, prob, feat = (predictions[k] for k in ['cls', 'prob', 'feat'])
            
            label_lists += list(data_dict['label'].cpu().detach().numpy())
            prediction_lists += list(predictions['prob'].cpu().detach().numpy())
            feature_lists += list(predictions['feat'].cpu().detach().numpy())
    
        # print(prediction_lists)
        y_pred = np.array(prediction_lists)
        threshold = 0.5
        y_pred = (y_pred >= threshold).astype(int)
        y_true = np.array(label_lists)
        # print(y_true)
        print(model_run)
        # Calculate and print the accuracy
        accuracy = accuracy_score(y_true, y_pred)
        print(f'Test Accuracy: {accuracy:.2f}')
        
        # Detailed classification report
        print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
        print(confusion_matrix(y_true, y_pred))
        

INFO:detectors.resnet34_bcos_detector:Load pretrained model successfully!
  0%|          | 0/1862 [00:00<?, ?it/s]

## Xception

In [35]:
# check performance of all model runs
weights_base_path = '/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/'
runs = [f'{weights_base_path}xception_2025-02-06-20-05-09/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-07-10-17-17/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-07-12-19-31/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-07-17-23-41/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-07-21-26-03/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-07-21-43-33/test/avg/ckpt_best.pth',
        f'{weights_base_path}xception_2025-02-08-09-13-00/test/avg/ckpt_best.pth']
                      

for model_run in runs:
    path = "./config/detector/xception.yaml"
    
    additional_args = {'test_batchSize': 12, 
                       'pretrained':model_run}
    config = load_config(path, additional_args=additional_args)

    test_data_loaders = prepare_testing_data(config)

    # prepare the model (detector)
    model_class = DETECTOR[config['model_name']]
    model = model_class(config)

    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Lists to store all predictions and true labels
    all_preds = []
    all_labels = []
    
    # Disable gradient calculation
    with torch.no_grad():
        key = list(test_data_loaders.keys())[0]
        # print(key)
        prediction_lists = []
        feature_lists = []
        label_lists = []
        for i, data_dict in tqdm(enumerate(test_data_loaders[key]), total=len(test_data_loaders[key])):#img_batch, label_batch in dataloader:
            img_batch, label_batch, landmark, mask = (data_dict[k] for k in ['image', 'label', 'landmark', 'mask'])
    
            # print(f"Batch of images shape: {img_batch.shape}")
            # device = "cpu"
            if 'label_spe' in data_dict:
                data_dict.pop('label_spe')  # remove the specific label
            data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
            # move data to GPU elegantly
            for key in data_dict.keys():
                if data_dict[key]!=None:
                    data_dict[key]=data_dict[key].to(device)#cuda()
    
    
            predictions = inference(model, data_dict)
            cls, prob, feat = (predictions[k] for k in ['cls', 'prob', 'feat'])
            
            label_lists += list(data_dict['label'].cpu().detach().numpy())
            prediction_lists += list(predictions['prob'].cpu().detach().numpy())
            feature_lists += list(predictions['feat'].cpu().detach().numpy())
    
        # print(prediction_lists)
        y_pred = np.array(prediction_lists)
        threshold = 0.5
        y_pred = (y_pred >= threshold).astype(int)
        y_true = np.array(label_lists)
        # print(y_true)
        print(model_run)
        # Calculate and print the accuracy
        accuracy = accuracy_score(y_true, y_pred)
        print(f'Test Accuracy: {accuracy:.2f}')
        
        # Detailed classification report
        print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
        print(confusion_matrix(y_true, y_pred))
        

INFO:detectors.xception_detector:Load pretrained model successfully!
100%|██████████| 1862/1862 [01:14<00:00, 24.98it/s]


/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/xception_2025-02-06-20-05-09/test/avg/ckpt_best.pth
Test Accuracy: 0.20
              precision    recall  f1-score   support

        Real       0.20      1.00      0.33      4478
        Fake       0.00      0.00      0.00     17864

    accuracy                           0.20     22342
   macro avg       0.10      0.50      0.17     22342
weighted avg       0.04      0.20      0.07     22342

[[ 4478     0]
 [17864     0]]


INFO:detectors.xception_detector:Load pretrained model successfully!
100%|██████████| 1862/1862 [01:14<00:00, 24.97it/s]


/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/xception_2025-02-07-10-17-17/test/avg/ckpt_best.pth
Test Accuracy: 0.20
              precision    recall  f1-score   support

        Real       0.20      1.00      0.33      4478
        Fake       0.00      0.00      0.00     17864

    accuracy                           0.20     22342
   macro avg       0.10      0.50      0.17     22342
weighted avg       0.04      0.20      0.07     22342

[[ 4478     0]
 [17864     0]]


INFO:detectors.xception_detector:Load pretrained model successfully!
100%|██████████| 1862/1862 [01:14<00:00, 24.94it/s]


...

/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/xception_2025-02-07-21-26-03/test/avg/ckpt_best.pth
Test Accuracy: 0.80
              precision    recall  f1-score   support

        Real       0.00      0.00      0.00      4478
        Fake       0.80      1.00      0.89     17864

    accuracy                           0.80     22342
   macro avg       0.40      0.50      0.44     22342
weighted avg       0.64      0.80      0.71     22342

[[    0  4478]
 [    0 17864]]


INFO:detectors.xception_detector:Load pretrained model successfully!
100%|██████████| 1862/1862 [01:14<00:00, 24.91it/s]


/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/xception_2025-02-07-21-43-33/test/avg/ckpt_best.pth
Test Accuracy: 0.80
              precision    recall  f1-score   support

        Real       0.00      0.00      0.00      4478
        Fake       0.80      1.00      0.89     17864

    accuracy                           0.80     22342
   macro avg       0.40      0.50      0.44     22342
weighted avg       0.64      0.80      0.71     22342

[[    0  4478]
 [    0 17864]]


INFO:detectors.xception_detector:Load pretrained model successfully!
100%|██████████| 1862/1862 [01:14<00:00, 24.90it/s]

/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/BWCluster/xception/logs/xception_2025-02-08-09-13-00/test/avg/ckpt_best.pth
Test Accuracy: 0.20
              precision    recall  f1-score   support

        Real       0.20      1.00      0.33      4478
        Fake       0.00      0.00      0.00     17864

    accuracy                           0.20     22342
   macro avg       0.10      0.50      0.17     22342
weighted avg       0.04      0.20      0.07     22342

[[ 4478     0]
 [17864     0]]
